<a href="https://colab.research.google.com/github/RobertoBarrosoLuque/SocialMobilityandSTEMEducation/blob/main/static_project_barrosoluquer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Roberto Barroso Luque
Data Visualization CAPP 30239

# **What is the relationship between STEM education outcomes and social mobility across the United States?**


In [ ]:
!pip install geopandas
!pip install gpdvega
!pip install mapclassify

import altair as alt
import seaborn as sns
import pandas as pd 
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from google.colab import files
import io
import json
from vega_datasets import data
uploaded = files.upload()

     |████████████████████████████████| 972kB 7.3MB/s 
     |████████████████████████████████| 14.8MB 293kB/s 
     |████████████████████████████████| 6.5MB 41.2MB/s 


Saving eighth-grade-math-proficiency.xlsx to eighth-grade-math-proficiency (2).xlsx
Saving eighth-grade-science-proficiency.xlsx to eighth-grade-science-proficiency (2).xlsx
Saving fourth-grade-math-proficiency.xlsx to fourth-grade-math-proficiency (2).xlsx
Saving fourth-grade-science-proficiency.xlsx to fourth-grade-science-proficiency (2).xlsx
Saving se-bachelors-degrees-per-1000.xlsx to se-bachelors-degrees-per-1000 (2).xlsx
Saving states_by_region.xlsx to states_by_region (2).xlsx
Saving table2_state_absmob_by_cohort.xls to table2_state_absmob_by_cohort (2).xls


In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# Set current directory
%cd "/gdrive/MyDrive/UChicago/DataViz/StaticProject"

Mounted at /gdrive
/gdrive/MyDrive/UChicago/DataViz/StaticProject


In [ ]:
# Load hex-states shapefile
hex_states = gpd.read_file("/gdrive/MyDrive/UChicago/DataViz/StaticProject/HexStates/HexStates/HexStates.shp")

# load all other files
to_skip = [0,1,2]
to_skip.extend(list(range(56,65)))

social_mobility = pd.read_excel(io.BytesIO(uploaded['table2_state_absmob_by_cohort.xls']))
states_by_region = pd.read_excel(io.BytesIO(uploaded['states_by_region.xlsx']))
states_by_region = states_by_region.dropna()

fips_code = social_mobility.drop_duplicates("state_fips")
states_by_region = states_by_region.merge(fips_code.loc[:,["state_fips","state_name"]],
                                          left_on='State Code', right_on="state_name")
states_by_region.columns = ["State", "State Code", "Region", "Division", "id", "state_name"]

hex_states = hex_states.merge(states_by_region.loc[:,["Region", "state_name", "id"]],
                              left_on="State_Abbr", right_on="state_name")

# load fourth grade indicators
fourth_grade_sci = pd.read_excel(io.BytesIO(uploaded['fourth-grade-science-proficiency.xlsx']), skiprows=to_skip)
fourth_grade_math = pd.read_excel(io.BytesIO(uploaded['fourth-grade-math-proficiency.xlsx']), skiprows=to_skip)

# load eight grade indicators
eighth_grade_sci = pd.read_excel(io.BytesIO(uploaded['eighth-grade-science-proficiency.xlsx']), skiprows=to_skip)
eighth_grade_math = pd.read_excel(io.BytesIO(uploaded['eighth-grade-math-proficiency.xlsx']), skiprows=to_skip)

# load indicators 
bachelors_1000 = pd.read_excel(io.BytesIO(uploaded['se-bachelors-degrees-per-1000.xlsx']), skiprows=to_skip, sheet_name="percent")

# Save most recent value for each indicator 
all_indicators_df = pd.DataFrame({"State":bachelors_1000["State"].values,
                                  "fourth_grade_science": fourth_grade_sci["2015"].values,
                                  "fourth_grade_math": fourth_grade_math["2019"].values,
                                  "eighth_grade_science": eighth_grade_sci["2015"].values,
                                  "eighth_grade_math": eighth_grade_math["2019"].values,
                                  "bachelors": bachelors_1000["2018"].values})
pop = data.population_engineers_hurricanes()
all_indicators_df = all_indicators_df.merge(pop.loc[:, ['state', 'id']],
                                            left_on='State',
                                            right_on='state').drop(columns=['state'])
states = alt.topo_feature(data.us_10m.url, 'states')
state_code = {
    'United States': "USA",
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

all_indicators_df["abrev_State"] = all_indicators_df["State"].apply(lambda x:
                                                                    state_code[x]
                                                                    )
all_indicators_df = all_indicators_df.merge(states_by_region.loc[:, ["State Code", "Region", "Division"]], left_on="abrev_State",
                                            right_on="State Code")

Set theme for visualizations

In [ ]:
def barrosoluquer_theme():
    """
    Function to specify theme of visualizations
    Code repurposed from: 
    https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602 """
   
    # Typography
    font = "Gill Sans" 
    labelFont = "Gill Sans" 
    sourceFont = "Noto Sansb"    # Axes
    # Axes
    axisColor = "#000000"
    gridColor = "#DEDDDD"    # Colors
    main_palette = ["#1FB8FC", 
                    "#B2B2B2",
                    "#000000", 
                    "#FDCC3F", 
                    "#B4F5A9"
                    "#4ED4B5", 
                    "#FF887F", 
                    "#FFBEFF", 
                   ]
    sequential_palette = ["#D5FFFF","#95F8FF","#70D3FC","#0099EA", 
                          "#005B97", "#3D478B", "#000027"]
    return {
        "config": {
            "title": {
                "fontSize": 24,
                "font": font,
                "anchor": "start", # equivalent of left-aligned.
                "fontColor": "#000000"
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                "domainWidth": 1,
                "grid": False,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0, 
                "tickColor": axisColor,
                "tickSize": 5, # default, including it just to show you can change it
                "titleFont": font,
                "titleFontSize": 14,
                "titlePadding": 10, # guessing, not specified in styleguide
                "title": "X Axis Title (units)", 
            },
            "axisY": {
                "domain": False,
                "grid": True,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0, 
                "ticks": False, # even if you don't have a "domain" you need to turn these off.
                "titleFont": font,
                "titleFontSize": 14,
                "titlePadding": 10, # guessing, not specified in styleguide
                "title": "Y Axis Title (units)", 
                # titles are by default vertical left of axis so we need to hack this 
                "titleAngle": 0, # horizontal
                "titleY": -10, # move it up
                "titleX": 18, # move it to the right so it aligns with the labels 
            },
            "range": {
                "category": main_palette,
                "diverging": sequential_palette,
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 14,
                "symbolType": "square", # just 'cause
                "symbolSize": 100, # default
                "titleFont": font,
                "titleFontSize": 12,
                "title": "", # set it to no-title by default
               # "orient": "top-right", # so it's right next to the y-axis
                "offset": 0, # literally right next to the y-axis.
            }
            }
    }
# register
alt.themes.register("barrosoluquer_theme", barrosoluquer_theme)
# enable
alt.themes.enable("barrosoluquer_theme")


# Set matplotlib specifications
plt.rc('font', size=16)          # controls default text sizes
plt.rc('axes', titlesize=16)     # fontsize of the axes title
plt.rc('axes', labelsize=16)    # fontsize of the x and y labels
plt.rc('xtick', labelsize= 20)    # fontsize of the tick labels
plt.rc('ytick', labelsize= 20)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)    # legend fontsize
plt.rc('figure', titlesize=24)  # fontsize of the figure title

In [ ]:
# Melt data frames
fourth_grade_sci = pd.melt(fourth_grade_sci, id_vars=['State'], value_vars=['2009', '2015'],
                           var_name='year', value_name='science_ach')
fourth_grade_sci = fourth_grade_sci.dropna()
fourth_grade_sci['science_ach'] = fourth_grade_sci['science_ach'].astype(int)
fourth_grade_sci['school_year'] = "4th"


fourth_grade_math = pd.melt(fourth_grade_math, id_vars=['State'], value_vars=['2000', '2019'],
                           var_name='year', value_name='math_ach')
fourth_grade_math = fourth_grade_math.dropna()
fourth_grade_math = fourth_grade_math[pd.to_numeric(fourth_grade_math['math_ach'], errors='coerce').notnull()]
fourth_grade_math['math_ach'] = fourth_grade_math['math_ach'].dropna().astype(int)
fourth_grade_math['school_year'] =  "4th"


eighth_grade_sci = pd.melt(eighth_grade_sci, id_vars=['State'], value_vars=['2009', '2015'],
                           var_name='year', value_name='science_ach')
eighth_grade_sci = eighth_grade_sci.dropna()
eighth_grade_sci['science_ach'] = eighth_grade_sci['science_ach'].astype(int)
eighth_grade_sci['school_year'] = "8th"


eighth_grade_math = pd.melt(eighth_grade_math, id_vars=['State'], value_vars=['2000', '2019'],
                           var_name='year', value_name='math_ach')
eighth_grade_math = eighth_grade_math.dropna()
eighth_grade_math = eighth_grade_math[pd.to_numeric(eighth_grade_math['math_ach'], errors='coerce').notnull()]
eighth_grade_math['math_ach'] = eighth_grade_math['math_ach'].dropna().astype(int) 
eighth_grade_math['school_year'] = "8th"

bachelors_copy = pd.melt(bachelors_1000, id_vars=['State'], value_vars=bachelors_1000.columns[1:],
                           var_name='year', value_name='stem_bachelors_1000')

math = pd.concat([fourth_grade_math, eighth_grade_math])
science = pd.concat([fourth_grade_sci, eighth_grade_sci])

math["state_code"] = math["State"].apply(lambda x:state_code[x])
science["state_code"] = science["State"].apply(lambda x:state_code[x])

math = math.merge(states_by_region.loc[:, ["State Code", "Region", "Division"]], left_on="state_code",
                                            right_on="State Code")
science = science.merge(states_by_region.loc[:, ["State Code", "Region", "Division"]], left_on="state_code",
                                            right_on="State Code")

science["pct_change"] = science.sort_values(by=["State", "year"]).groupby("State")["science_ach"].pct_change()
math["pct_change"] = math.sort_values(by=["State", "year"]).groupby("State")["math_ach"].pct_change()

science.fillna(0, inplace=True)
math.fillna(0, inplace=True)

science["change"] = science["pct_change"].apply(lambda row: "Increase" if row>=0 else "Decrease")
math["change"] = math["pct_change"].apply(lambda row: "Increase" if row>=0 else "Decrease")

In [ ]:
science_avg = science.groupby(by=["State",'school_year']).mean().reset_index()
science_avg["code"] = science_avg["State"].apply(lambda x:state_code[x])

science_avg = science_avg.merge(states_by_region.loc[:, ["State Code", "Region", "Division"]], left_on="code",
                                right_on="State Code")


math_avg = math.groupby(by=["State", "school_year"]).mean().reset_index()
math_avg["code"] = math_avg["State"].apply(lambda x:state_code[x])

math_avg = math_avg.merge(states_by_region.loc[:, ["State Code", "Region", "Division"]], left_on="code",
                          right_on="State Code")

# **Visualizing STEM education outcomes at various levels**

In the first three figures I look at the STEM achievement of all states across threee different education levels: 4th grade achievement, 8th grade achievement and bachelors level achievemnt. 




## Figure 1

In [ ]:
math2use = math.loc[(math.State != "Alaska") & (math.State != "District of Columbia"), :]

# Make plot for Science outcomes
lines_sci = alt.Chart().mark_line(color="black", size=3).encode(
    x=alt.X('science_ach:Q', title="% met standard"),
    y=alt.Y('school_year:N', axis=alt.Axis(labels=False)),
    detail='school_year:N'
    )
points_sci = alt.Chart().mark_point(size=150,
        opacity=1,
        filled=True).encode(
    x=alt.X('science_ach:Q', title="% met standard", axis=alt.Axis(grid=True, orient="top", gridColor ="black",
                                                                   labelPadding=15)),
    y=alt.Y('school_year:N', axis=alt.Axis(labels=False, gridColor ="black"),scale=alt.Scale(padding=0)),
    color=alt.Color('year:O', scale=alt.Scale(domain=['2009', '2015'],range= ["#B2B2B2","#4CCC3B"]),
                    legend=alt.Legend(orient="left", offset=10), title="year"),
    shape=alt.Shape('school_year:N',scale=alt.Scale(domain=['4th', '8th'],range=["diamond", "circle"]),
                    legend=alt.Legend(orient="left"), title="grade")
)
science_chart = alt.layer(
    lines_sci,
    points_sci,
    data=science,
).facet(title=alt.TitleParams("Science",dx=120,dy=-10),
        row=alt.Row(
        'state_code:N',
        title=None,
        header=alt.Header(labels=False),
        sort=alt.EncodingSortField(field='state_code', order='ascending')),
        spacing={"row":10}
)

# Make plot for Mathematics outcomes
lines_math = alt.Chart().mark_line(color="black",size=3).encode(
    x=alt.X('math_ach:Q', title="% met standard"),
    y=alt.Y('school_year:N', axis=alt.Axis(labels=True),scale=alt.Scale(padding=0)),
    detail='school_year:N'
    )
points_math = alt.Chart().mark_point(size=150,
        opacity=1,
        filled=True).encode(
    x=alt.X('math_ach:Q', title="% met standard", axis=alt.Axis(grid=True, orient="top", gridColor ="black",
                                                                labelPadding=15)),
    y=alt.Y('school_year:N', axis=alt.Axis(labels=False, gridColor ="black"),scale=alt.Scale(padding=0)),
    color=alt.Color('year:O', scale=alt.Scale(domain=['2000', '2019'],range= ["#B2B2B2","#1FB8FC"]),
                    legend=alt.Legend(orient="right", offset=10), title="year"),
    shape=alt.Shape('school_year:N',scale=alt.Scale(domain=['4th', '8th'],range=["diamond", "circle"]),
                      legend=None, title="grade")
)
math_chart = alt.layer(
    lines_math,
    points_math,
    data=math2use,
).facet(title=alt.TitleParams("Mathematics",dx=100, dy=-10),
        row=alt.Row(
        'state_code:N',
        header=alt.Header(labelAngle=0, labelAlign="left",
                          labelAnchor="middle",labelFontSize=12),
        title=" ",
        sort=alt.EncodingSortField(field='state_code', order='ascending')),
        spacing={"row":10}
)

# Concatenate charts together
last_chart = alt.ConcatChart(
    concat=[science_chart, math_chart], columns=2
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
).configure_concat(
    spacing=-25
).configure_title(fontSize=20).configure_view(stroke="transparent").configure_axisY(
    labelPadding=-205, 
    labelAlign='left',
    labelFontSize=10)
last_chart.properties(title = alt.TitleParams(
                      ["Percent of 4th and 8th grade public school",
                       "students who met the subject standards"],
                       subtitle = ["Source: National Science Foundation",""],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=20,
                       dx=30),
)

alt.ConcatChart(...)

The figure above shows how the percent of public school students who score at or above the national standard in both science and math varies across states and how it has changed across time (from 2009 to 2015 for science and 2000 to 2019 for mathematics). Two things are worth highlighting from this chart. The first is the variance across states for both disciplines, While some states such as Missisipi, DC, Alabama have rates of 30% or less proficiency other states such as Vermont and New Jersey have rates approximating 50% proficiency. Second, for the vast majority of states we see improvements across time periods in their proficiency rates for both disciplines and both years shown.  

In [ ]:
eighth_sci = eighth_grade_sci
eighth_sci["indicator"] = "Science"
eighth_sci.columns = ["State", "year", "achievement", 'school_year',"indicator"]

eighth_math = eighth_grade_math
eighth_math["indicator"] = "Mathematics"
eighth_math.columns = ["State", "year", "achievement",  'school_year', "indicator"]


df_to_use = pd.concat([eighth_sci,eighth_math])
df_to_use = df_to_use.loc[df_to_use.State != "United States"]

## Figure 2

In [ ]:
palette = ["#00D2D5", "#008CF9", "#5DDF84", "#009050"]

region_map = alt.Chart(states).mark_geoshape(stroke='white').encode(
    color=alt.Color("Region:N", scale=alt.Scale(domain=["Midwest", "Northeast",
                                                          "South", "West"], range= palette), 
                    legend=alt.Legend(title="Region", orient="top"))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(states_by_region, 'id', ['Region'])
).properties(
    width=200,
    height=200
).project(
    type='albersUsa'
)


science_strip = alt.Chart(science_avg,title=alt.TitleParams("Science",dx=165),
                           height=500, width=80).mark_point(size=200, filled=True).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('science_ach:Q', scale=alt.Scale(domain=[14,48]), axis=alt.Axis(grid=False),
            title="% students met proficiency standard"),
    color=alt.Color('Region:N', scale = alt.Scale(domain=["Midwest", "Northeast",
                                                          "South", "West"], range=palette),
                    legend=None),
    shape=alt.Shape('school_year:N',scale=alt.Scale(domain=['4th', '8th'],range=["diamond", "circle"]),
                    legend=alt.Legend(orient="left"), title="grade"),
        column=alt.Column(
        'Region:N',
        header=alt.Header(
            titleOrient='top',
            labelOrient='bottom',
            labelAlign='center',
            labelPadding=10,
        ),title="",
    )).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
)

math_strip =  alt.Chart(math_avg, title=alt.TitleParams("Mathematics",dx=100),
                           height=500, width=80).mark_point(size=200,filled=True).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('math_ach:Q', scale=alt.Scale(domain=[14,48]), axis=None),
    color=alt.Color('Region:N', scale = alt.Scale(domain=["Midwest", "Northeast",
                                                          "South", "West"],
                                                   range=palette),
                    legend=None),
    shape=alt.Shape('school_year:N',scale=alt.Scale(domain=['4th', '8th'],range=["diamond", "circle"]),
                    legend=alt.Legend(orient="left"), title="grade"),
        column=alt.Column(
        'Region:N',
        header=alt.Header(
            titleOrient='top',
            labelOrient='bottom',
            labelAlign='center',
            labelPadding=10,
        ),title="",
    )).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
)

math_strip = alt.hconcat(math_strip, region_map).resolve_legend(color='independent')

alt.concat(science_strip, math_strip, spacing=100).properties(title = alt.TitleParams(
                      ["Average of students who met or exceeded",
                       "STEM proficiency standard by region"],
                       subtitle = ["Science is average for 2009 & 2015 values",
                                   "Mathematics is average for 2000 & 2019 values",
                                   "Each point is for a single state",
                                   "",
                                   "Source: National Science Foundation"],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=20, dx=-150)).configure_facet(
    spacing=1
).configure_view(
    stroke=None
).configure_axisY(
    titleAngle=-90, titleY = 250, titleX = -40
).resolve_legend(color='independent')

alt.ConcatChart(...)

Figure 2 makes the variance of profficiency standards for both Mathematics and Science across regions clear to view. This figure is averaging for each state across both 4th and 8th grade proficiency percentages. This visualization helps illustrate the vast STEM outcome inequality across the country. The Northeast and Midwest have, on average, better outcomes for both fields when compared with the South and West. Furthermore, both the South and West regions have much more variance, with some states their performing well and other performing badly.


## **Visualizing bachelors degrees in STEM per 1000 residents**

In [ ]:
bachelors_copy["pct_change"] = bachelors_copy.sort_values(by=["State", "year"]).groupby("State")["stem_bachelors_1000"].pct_change()
bachelors_copy["pct_change"] = bachelors_copy["pct_change"].values * 100
bachelors_copy.fillna(0, inplace=True)
bachelors_copy.head()

max_val = max(bachelors_copy.stem_bachelors_1000.values)
min_val = min(bachelors_copy.stem_bachelors_1000.values)

## Figure 3

In [ ]:
palette = ["#0059A1","#0086F3","#66B5FF","#FFB832", "#E36A67","#FF3E52", "#D50000"]
palette.reverse()


alt.Chart(bachelors_copy).mark_point(
    filled=True,
    size=200,
    opacity=1
    ).encode(
    y= alt.Y('State',axis=alt.Axis(grid=True, gridColor="#555555", gridDash=[1])),
    x='year',
    color=alt.Color('stem_bachelors_1000',scale=alt.Scale(range=palette), bin=alt.Bin(maxbins=6),
                    legend=alt.Legend(title="Degrees per 1,000 ⟶", orient="left", titleOrient="left")),
    shape=alt.Shape(
         'pct_change:O',bin=alt.BinParams(maxbins=2),
         scale = alt.Scale(range=["triangle-down", "triangle-up"],zero=True),
         legend=alt.Legend(title=["% change" ,"year earlier"], labelFontSize=14, orient="left",
                           titleOrient="bottom",))
).properties(title = alt.TitleParams(
                      ["Number  of S&E Degrees Conferred per 1,000 Individuals 18–24 Years Old"],
                       subtitle = ["Source: National Science Foundation",""],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=20),
                       width=800, 
                       height=900
             ).configure_legend(symbolSize=1000, 
                                symbolStrokeWidth=10
                                ).configure_axisY(labelFontSize=13)



alt.Chart(...)

Figure 3 has a lot of information! The plot shows th number  of S&E degrees conferred per 1,000 individuals (18–24 years old) for all states (rows) and across the past 18 years (columns). Furthermore, the number of S&E degrees conferred for each state on each year is encoded by color. The darker color means more degrees where conferred and viceversa for the brighter colors. Finally, we can also visualize the change from year to year by looking at the direction of the triangles, with triangles pointing dows signifying there was a fall in degrees conferred from last year and the opposite for triangles pointing up.

# **Relationship among different STEM attainmente indicators**

In [ ]:
from sklearn.linear_model import LinearRegression
linear_regressor = LinearRegression()

In [ ]:
all_indicators_clean = all_indicators_df.dropna()
all_indicators_melt = pd.melt(all_indicators_clean, id_vars=['State'], value_vars=all_indicators_clean.columns[1:]
                             , value_name='achievement')
all_indicators_melt = all_indicators_melt.dropna()

## Figure 4

In [ ]:
palette = ["#00D2D5", "#008CF9", "#5DDF84", "#009050"]

region_map = alt.Chart(states).mark_geoshape(stroke='white').encode(
    color=alt.Color("Region:N", scale=alt.Scale(domain=["Midwest", "Northeast",
                                                          "South", "West"], range= palette), 
                    legend=alt.Legend(title="Region", orient="right"))
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(states_by_region, 'id', ['Region'])
).properties(
    width=150,
    height=150
).project(
    type='albersUsa'
)

base = alt.Chart().mark_point(size=100, filled=True,opacity=0.60).encode(
    color=alt.Color("Region:N", scale=alt.Scale(domain=["Midwest", "Northeast",
                                                        "South", "West"], range= palette),
                    legend=None)
).properties(
    width=120,
    height=120
)

chart = alt.vconcat(data=all_indicators_df, spacing=15)

all_encodings = ['fourth_grade_science:Q', 'fourth_grade_math:Q',
                 'eighth_grade_science:Q', 'eighth_grade_math:Q', 'bachelors:Q']

title_dict = {'fourth_grade_science:Q':'4th-grade-Science',
              'fourth_grade_math:Q':'4th-grade-math',
              'eighth_grade_science:Q':'8th-grade-Science',
              'eighth_grade_math:Q':'8th-grade-math',
              'bachelors:Q':'Bachelors'}

pairs_done = []

for y_encoding in all_encodings[::-1]:
    row = alt.hconcat(spacing=15)
    y_max = max(all_indicators_df[y_encoding[:-2]].values) + 2
    y_min = min(all_indicators_df[y_encoding[:-2]].values) - 2
    for x_encoding in all_encodings:

        if x_encoding == y_encoding or (x_encoding, y_encoding) in pairs_done or  (y_encoding, x_encoding) in pairs_done:
          continue
        
        pairs_done.append((y_encoding, x_encoding))
        
        x_max = max(all_indicators_df[x_encoding[:-2]].values) + 2
        x_min = min(all_indicators_df[x_encoding[:-2]].values) - 2

        new_chart = base.encode(
            x=alt.X(x_encoding, scale=alt.Scale(domain=[x_min, x_max], zero=False),
                    axis=alt.Axis(title=title_dict[x_encoding])),
            y=alt.Y(y_encoding, scale=alt.Scale(domain=[y_min, y_max], zero=False),
                    axis=alt.Axis(title=title_dict[y_encoding], grid=False))
        )

        # Perform regression

        X = all_indicators_df[x_encoding[:-2]].values
        Y = all_indicators_df[y_encoding[:-2]].values

        if any(np.isnan(X)):
          idx = ~np.isnan(X)
        elif any(np.isnan(Y)):
          idx = ~np.isnan(Y)
        else:
          idx = ~np.isnan(Y)
        X = X[idx].reshape(-1, 1)
        Y = Y[idx].reshape(-1,1)
  
        linear_regressor.fit(X, Y)  # perform linear regression
        Y_pred = linear_regressor.predict(X)  # make predictions


        reg_line = alt.Chart(pd.DataFrame({'x':X[:,0], 'y': Y_pred[:,0]})).mark_line(color= '#b7bcc0').encode(x='x', y='y')

                
        row |= (new_chart + reg_line)

    chart &= row


final_chart = alt.concat(chart, region_map,  center=True).configure_concat(spacing=0).properties(title = alt.TitleParams(
                      ["Relationship across STEM indicators"],
                       subtitle = ["Bachelors: STEM degrees per 1000 individuals 18–24 years old",
                                   "Fourth grade: % met or exceeded the domain proficiency standard",
                                   "Eighth grade: % met or exceeded the domain proficiency standard",
                                   "", 
                                   "Source: National Science Foundation",""],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=10)).resolve_scale(x="independent",
                                                          y="independent").resolve_legend(color='independent')

final_chart.configure_axisX(titleY=18).configure_view(
    strokeWidth=1
).configure_axisY(
    titleAngle=-90, 
    titleY = 60, 
    titleX = -20,
    labelFontSize=12,
    grid=True)
    

alt.ConcatChart(...)

Figure 4 allows us to, albeit superficially, understand the relationship among different STEM indicators studied. Some clear patterns emerge from this visualization. First, there is a strong linear relationship when looking at the same discipline for different education levels (i.e. fourth grade math is positively correlated eighth grade math). The same relationship, while a little weaker is still present when comparing across disciplines (i.e. science vs math). Second, while there is still a positive trend between bachelor attainment and fourth and eighth grade proficiency, the relationships seem much more tenuous.

# **Visualizing Social Mobility by State**

In [ ]:
mobility_1980 = social_mobility.loc[(social_mobility["cohort"] == 1980), :]
first_last_mobility = first_last_mobility.loc[:,:] = social_mobility.loc[(social_mobility["cohort"] == 1940) | (social_mobility["cohort"] == 1980), :]
first_last_mobility.loc[:, "pct_change"] = first_last_mobility.sort_values(by=["state_name", "cohort"]).groupby("state_name")["cohort_mean"].pct_change()

largest_drop = min(first_last_mobility["pct_change"].dropna().values)
smallest_drop = max(first_last_mobility["pct_change"].dropna().values)
worst_best = first_last_mobility.loc[(first_last_mobility["state_name"] == "AK") | (first_last_mobility["state_name"] == "DC"), :]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

## Figure 5

In [ ]:
first_last_mobility.state_name.unique()
line_colors = ["#B0B0B0"]*len(first_last_mobility.state_name.unique())
line_colors[2] = '#00D2D5'

In [ ]:
chart1 = alt.layer(
    data=first_last_mobility, width=400, height=600,title = alt.TitleParams("Social mobility for all states",dx=165)
).resolve_scale(color="independent")

chart1 += alt.Chart().mark_line(opacity=.6).encode(
    x='cohort:N',
    y=alt.Y('cohort_mean:Q',  title="Probability",
            scale=alt.Scale(domain=[.35, 1])),
    detail='state_name:N'
)    
chart1 += alt.Chart().mark_point(
    size=100,
    opacity=1,
    filled=True
).encode(
    x=alt.X('cohort:N', title="Birth year",axis=alt.Axis(grid=False)),
    y=alt.Y('cohort_mean:Q'),
    color=alt.Color('cohort:O',
        scale=alt.Scale(
            domain=['1940', '1980'],
            range= ["#818181","#818181"],
        ),
        legend=None
    )
)
# HIGHLIGHT AK
chart1 += alt.Chart(data=first_last_mobility.loc[first_last_mobility.state_name=="AK"]
                    ).mark_line(size=5).encode(x='cohort:N',
                        y=alt.Y('cohort_mean:Q',  title="",
                                scale=alt.Scale(domain=[.35, 1])),
                        color = alt.Color("state_name:N", scale=alt.Scale(domain=["AK"], range=["#00D2D5"]),
                                          legend=None))
chart1 += alt.Chart(data=first_last_mobility.loc[first_last_mobility.state_name=="AK"]
                    ).mark_point(size=200,opacity=1,filled=True).encode(x='cohort:N',
                        y=alt.Y('cohort_mean:Q',  title="",
                                scale=alt.Scale(domain=[.35, 1])),
                        color = alt.Color("state_name:N", scale=alt.Scale(domain=["AK"], range=["#00D2D5"]),
                                          legend=None))
#HIGHLIGHT DC
chart1 += alt.Chart(data=first_last_mobility.loc[first_last_mobility.state_name=="DC"]
                    ).mark_line(size=5).encode(x='cohort:N',
                        y=alt.Y('cohort_mean:Q',  title="",
                                scale=alt.Scale(domain=[.35, 1])),
                        color = alt.Color("state_name:N", scale=alt.Scale(domain=["DC"], range=["#5DDF84"]),
                                          legend=None))
chart1 += alt.Chart(data=first_last_mobility.loc[first_last_mobility.state_name=="DC"]
                    ).mark_point(size=200,opacity=1,filled=True).encode(x='cohort:N',
                        y=alt.Y('cohort_mean:Q',  title="",
                                scale=alt.Scale(domain=[.35, 1])),
                        color = alt.Color("state_name:N", scale=alt.Scale(domain=["DC"], range=["#5DDF84"]),
                                          legend=None))

DC = alt.Chart(data=first_last_mobility.loc[first_last_mobility.state_name=="DC"],
                   width=100, height=250,
                   title=alt.TitleParams("D.C.", dx=50)).mark_bar().encode(
    x=alt.X('cohort:N',title="Birth year"),
    y=alt.Y('cohort_mean:Q'),
    color=alt.Color("state_name:N", scale=alt.Scale(domain=["DC"], range=['#5DDF84']),
                     legend=None))

AK = alt.Chart(data=first_last_mobility.loc[first_last_mobility.state_name=="AK"],
                   width=100, height=250,
                   title=alt.TitleParams("AK", dx=50)).mark_bar().encode(
    x=alt.X('cohort:N',title="Birth year"),
    y=alt.Y('cohort_mean:Q'),
    color=alt.Color("state_name:N", scale=alt.Scale(domain=["AK"], range=['#00D2D5']),
                     legend=None))

chart2= alt.vconcat(DC,AK).resolve_scale(
    color='independent'
)

final_chart = alt.ConcatChart(concat=[chart1, chart2]).resolve_scale(
    color='independent'
).configure_view(
    stroke=None,
).configure_concat(
    spacing=27
).configure_title(fontSize=20).configure_axisY(
    titleAngle=-90, titleY = 400, titleX = -40
)

final_chart.properties(title = alt.TitleParams(
                      ["Social mobility on the downfall"],
                       subtitle = ["Social mobility measured as probability that a child earns more than their",
                                   "parents between ages 25-35 by state and cohort.", "",
                                   "Source: Opportunity Insights Project, Harvard University",""],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=20))

alt.ConcatChart(...)

The figure above shows the very stark reality of social mobility decline in the United States. All people born in 1940, regardless of their state, had at least a .85 change of earning more than their parents by age 25-34. For the cohort born in 1980, the majority of people had less than a .60 change of earning more than their parents. 

# **Looking at the correlation of social mobility with various STEM indicators**

In [ ]:

stem_mobility = first_last_mobility.loc[first_last_mobility.cohort==1980].merge(all_indicators_df, left_on="state_name", right_on="abrev_State")

## Figure 6

In [ ]:
stem_mob_interest = stem_mobility.loc[:, ["cohort_mean", 'fourth_grade_science', 'fourth_grade_math', 'eighth_grade_science',
                            'eighth_grade_math', 'bachelors']]

stem_mob_interest.columns = ["Social Mobility","4th-grade-science","4th-grade-math",
                             "8th-grade-science", "8th-grade-math",  "S&E bachelors"]


l1 = sorted(stem_mob_interest.corr().columns.to_list()) # sorting is needed to make sure the final correlation maps looks like an upper or lower triangle
l2 = l1.copy()
rows = []

for e1 in l1:
    for e2 in l2:
        rows.append([e1,e2, stem_mob_interest.corr().loc[e1,e2]])
    l2.remove(e1)

# Create dataframe from list of rows
newdf = pd.DataFrame(rows,columns=['variable','variable2','correlation'])

# If you want to remove the diagonals
newdf = newdf[newdf['variable']!=newdf['variable2']]

newdf["correlation"] = newdf["correlation"].round(2)

In [ ]:
palette = ["#41FFFF","#00CCD9","#00A7E4","#006DD6", 
            "#005B97", "#3D478B", "#000027"]
color_scale = alt.Scale(range= palette)

base = alt.Chart(newdf, height=600, width=600).encode(
    x=alt.X('variable:N',axis=alt.Axis(labelAngle=-45, grid=False)),
    y=alt.Y('variable2:N',axis=alt.Axis(grid=False)))

# Text layer with correlation labels
# Colors are for easier readability
text = base.mark_text().encode(
    text=alt.Text('correlation'),
    color=alt.condition(
        alt.datum.correlation < 0.5, 
        alt.value('black'),
        alt.value('white')
    )
)

# The correlation heatmap itself
cor_plot = base.mark_rect().encode(
    color=alt.Color('correlation:Q', scale=color_scale,
                   legend=alt.Legend(title= "Correlation⟶", orient="right", titleOrient="left", padding=0))
)

final = cor_plot + text

final.resolve_legend(color='independent').properties(title = alt.TitleParams(
                      ["STEM education is important but not sufficient for social mobility"],
                       subtitle = ["Pairwise pearson-correlation between STEM education outcomes and",
                                   "social mobility indicator using most recently available data for all states.",
                                  "",
                                   "Source: National Science Foundation",
                                   "Opportunity Insights Lab, Harvard University"],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=20)).configure_view(stroke=None)

alt.LayerChart(...)

From the above correlation matrix we can see that while positively correlated with all STEM education attainment indicators, our social mobility index is most correlated with S&E Bachelors per 1000 18-24 residents per state. There are a few conjectures we can make for this correlation. Logically, it is not surprising that the number of S&E bachelors degrees are highly correlated with the probability that kids born in the 1980s will have a higher income than their parents.

Below we will see how this relationship plays out state by state.

## Figure 7

In [ ]:
hex_states.columns

Index(['State', 'State_Abbr', 'geometry', 'Region', 'state_name', 'id'], dtype='object')

In [ ]:
stem_mobility["bachelors_diff_mean"] = (stem_mobility["bachelors"] - stem_mobility["bachelors"].mean())/stem_mobility["bachelors"].std(ddof=0)
stem_mobility["mobility_diff_mean"] = (stem_mobility["cohort_mean"] - stem_mobility["cohort_mean"].mean())/stem_mobility["cohort_mean"].std(ddof=0)

stem_mobility["8th_science_diff_mean"] = (stem_mobility["eighth_grade_science"] - stem_mobility["eighth_grade_science"].mean())/stem_mobility["eighth_grade_science"].std(ddof=0)
stem_mobility["4th_science_diff_mean"] = (stem_mobility["fourth_grade_science"] - stem_mobility["fourth_grade_science"].mean())/stem_mobility["fourth_grade_science"].std(ddof=0)


stem_mobility["bachelors_diff_mean"] = stem_mobility["bachelors_diff_mean"].apply(lambda row: 1 if row > 0 else -1)
stem_mobility["mobility_diff_mean"] = stem_mobility["mobility_diff_mean"].apply(lambda row: 1 if row > 0 else -1)

stem_mobility["8th_diff_mean"] = stem_mobility["8th_science_diff_mean"].apply(lambda row: 1 if row > 0 else -1)
stem_mobility["4th_diff_mean"] = stem_mobility["4th_science_diff_mean"].apply(lambda row: 1 if row > 0 else -1)

of_interest = stem_mobility.loc[:, ['cohort_mean', 'fourth_grade_science', 'fourth_grade_math',
                                    'eighth_grade_science', 'eighth_grade_math', 'bachelors', 'state_name',
                                    'bachelors_diff_mean', 'mobility_diff_mean','8th_diff_mean', '4th_diff_mean']]
hex_states = hex_states.merge(of_interest, on="state_name", how="inner")

In [ ]:
hex_states['centroid_lon'] = hex_states['geometry'].centroid.x
hex_states['centroid_lat'] = hex_states['geometry'].centroid.y

choro_json = json.loads(hex_states.to_json())
choro_data = alt.Data(values=choro_json['features'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [ ]:
def gen_map(choro_data, color_column, 
            title, legend_title, color_scale, maxbin=6,
            format=None):
    '''Generates DC ANC map with population choropleth and ANC labels'''
 
    # Add Base Layer

    if not legend_title:

        base = alt.Chart(choro_data, title = title).mark_geoshape(
            stroke='black',
            strokeWidth=1
        ).encode(
        ).properties(
        width=400,
        height=400
        )
        # Add Choropleth Layer
        choro = alt.Chart(choro_data).mark_geoshape(
            stroke='black'
        ).encode(
            color = alt.Color(color_column, 
                              scale=color_scale,
                              bin=alt.Bin(maxbins=maxbin),
                              type='quantitative', 
                              legend=None))
    elif not format:
        base = alt.Chart(choro_data, title = title).mark_geoshape(
            stroke='black',
            strokeWidth=1
        ).encode(
        ).properties(
        width=400,
        height=400
        )
        # Add Choropleth Layer
        choro = alt.Chart(choro_data).mark_geoshape(
            stroke='black'
        ).encode(
            color = alt.Color(color_column, 
                              scale=color_scale,
                              bin=alt.Bin(maxbins=maxbin),
                              type='quantitative', 
                              legend=alt.Legend(orient="bottom", title= legend_title,labelFontSize=13,
                                                symbolSize=500,
                                                symbolStrokeWidth=3,
                                                titleFontSize=14,
                                                titleLimit=250)))
    else:
        base = alt.Chart(choro_data, title = title).mark_geoshape(
            stroke='black',
            strokeWidth=1
        ).encode(
        ).properties(
        width=400,
        height=400
        )
        # Add Choropleth Layer
        choro = alt.Chart(choro_data).mark_geoshape(
            stroke='black'
        ).encode(
            color = alt.Color(color_column, 
                              scale=color_scale,
                              bin=alt.Bin(maxbins=maxbin),
                              type='quantitative', 
                              legend=alt.Legend(orient="bottom", title= legend_title, format=format,
                                                labelFontSize=13,symbolSize=500,
                                                symbolStrokeWidth=3,
                                                titleFontSize =14,
                                                titleLimit=250)))
    # Add Labels Layer
    labels = alt.Chart(choro_data).mark_text(fontSize=12, baseline='top', color="white",fontWeight='bold',
                                             
     ).properties(
        width=400,
        height=400
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text=alt.Text('properties.state_name:O'),
         size=alt.value(10),
     )

    final =  base + choro + labels

    return final

In [ ]:
palette = ["#8BE4EF","#00CCD9","#00A7E4","#006DD6", 
           "#005B97", "#3D478B", "#000027"]
color_scale = alt.Scale(range= palette)

chart1 = gen_map(choro_data, "properties.bachelors", title="Bachelors degrees in Science and Engineering."
                 ,legend_title="S&E Degrees per 1000 people ⟶", color_scale=color_scale)

chart2 = gen_map(choro_data, "properties.cohort_mean", title="Probability of earning more than your parents between ages 25-35"
                 ,legend_title="% earn more than parents ⟶", color_scale=color_scale, format="%")



alt.hconcat(chart2, chart1).resolve_scale(
    color='independent'
).configure_view(
    stroke=None,
).configure_concat(
    spacing=150
).configure_title(fontSize=18).properties(title = alt.TitleParams(
                      ["A complicated relationship"],
                       subtitle = ["Not all states with high S&E bachelors attainment",
                                   "have high probability that a child earns more than their parents","",
                                   "Source: National Science Foundation",
                                   "Opportunity Insights Lab, Harvard University",""],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=10))


alt.HConcatChart(...)

In figure 7 we can see in more detail how the relationship between our chosen social mobility index and STEM bachelors attainment varies throughout different states. There does seem to be a relationship with states with higher S&E attainment tend to have higher rates of social mobility. Nonetheless the strength of such relationship is not immedietely clear from this plot. 

## Figure 8

In [ ]:
palette = ['#00CF6A','#4D88C8']
color_scale = alt.Scale(range= palette)


chart1 = gen_map(choro_data, "properties.bachelors_diff_mean", title="S&E Degrees per 1000 individuals 18-24 years old."
                 ,legend_title=None, color_scale=color_scale, maxbin=2)

chart2 = gen_map(choro_data, "properties.mobility_diff_mean", title="Probability of earning more than your parents between ages 25-35"
                 ,legend_title="Worse than avg  |  Better than avg",color_scale=color_scale, maxbin=2)


chart3 = gen_map(choro_data, "properties.4th_diff_mean", title="Fourth grade science public school students % proficient            '"
                 ,legend_title=None, color_scale=color_scale, maxbin=2)
chart4 = gen_map(choro_data, "properties.8th_diff_mean", title= "Eighth grade science public school students % proficient"
                 ,legend_title=None,color_scale=color_scale, maxbin=2)


row1=alt.hconcat(chart2, chart1).resolve_scale(
    color='independent'
)
row2=alt.hconcat(chart3, chart4).resolve_scale(
    color='independent'
)

alt.vconcat(row1,row2).resolve_scale(
    color='independent'
).configure_view(
    stroke=None,
).configure_concat(
    spacing=50
).configure_title(fontSize=20).properties(title = alt.TitleParams(
                      ["Better or worse than the rest?: How do states perform relative to the average?"],
                       subtitle = ["States with better than average social mobility do not necessarily have better than average STEM outcomes.","",
                                   "Source: National Science Foundation", 
                                   "Opportunity Insights Project, Harvard University",
                                    "", ""],
                       fontSize = 24,
                       anchor="middle",
                       subtitlePadding=10))


alt.VConcatChart(...)

In figure 8 by using z-scores to find the difference from the mean for bachelors S&E attainment as well as social mobility we can more clearly see which states perfom below the national mean and which ones perform above. What is somewhat clear is that states that perform close to or below the mean for bachelors S&E attainment also tend to have social mobility indeces close to or below the mean. 